In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [3]:
num_out = 2
class label_model_CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=30, kernel_size=(5, 5), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(5, 5), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(3, 3), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
            nn.Conv2d(in_channels=30, out_channels=30, kernel_size=(3, 3), stride=(1, 1)),
            nn.BatchNorm2d(30),
            nn.Sigmoid(),
        )
        self.flatten = nn.Flatten()
        self.Linear = nn.Linear(43320, num_out)

    def forward(self, x):
        x = self.layers(x)
        x = self.flatten(x)
        x = self.Linear(x)
        return x
def make_dir_try(dir):
    try:
        os.makedirs(dir, exist_ok=True)
        print("Directory '%s' created successfully" % dir)
    except OSError as error:
        print("Directory '%s' can not be created" % dir)

In [5]:
PS_labels = np.load('../data/Processed_pattern_stimuli/labels.npy')
def simple_index (x):
    if x==3:
        return 0
    else:
        return 1

PS_labels = np.array([simple_index(x) for x in PS_labels])
PS_labels_all = np.concatenate((PS_labels, PS_labels, PS_labels, PS_labels), 0)

PS_set1 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_1.npy')
PS_set2 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_2.npy')
PS_set3 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_3.npy')
PS_set4 = np.load('../data/Processed_pattern_stimuli/crop_100_resize_4.npy')
PS_imgs_all = np.concatenate((PS_set1, PS_set2, PS_set3, PS_set4), 0)
all_PS_sets = [PS_imgs_all, PS_set1, PS_set2, PS_set3, PS_set4]
all_PS_labels = [PS_labels_all, PS_labels, PS_labels, PS_labels, PS_labels]

all_PS_sets = [np.reshape(p, (p.shape[0],1,50, 50)) for p in all_PS_sets]

all_PS_sets = [torch.tensor(p, dtype=torch.float) for p in all_PS_sets]

batchsize = 100
for set_id, (set, labels) in enumerate(zip(all_PS_sets,all_PS_labels)):
    train_labels = torch.tensor(labels, dtype=torch.long)
    dataset = TensorDataset(set, train_labels)
    loader = DataLoader(dataset, batch_size=batchsize, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = label_model_CNN()
    model = model.to(device)
    model.load_state_dict(torch.load('A:/school/College_Junior/research/CNN_Tang_project/saved_models/PS_labeling_model_CNN_CV' + str(set_id)))
    model.eval()
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        percent = 0
        losses = []
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y).detach().cpu().numpy()
            losses.append(loss)
            predict = torch.argmax(out, 1)
            for pp,yy in zip(predict,y):
                if pp == yy:
                    percent += 1
        percent /= set.shape[0]
        print(percent)
        print(np.mean(np.array(loss)))
        print('next')

0.9999736842105263
0.00028535505
next
0.9997894736842106
0.00036881707
next
0.999578947368421
0.0131967375
next
1.0
3.1445938e-06
next
1.0
8.404595e-06
next


In [ ]:
#sites = [ 'm1s1', 'm1s2', 'm1s3','m2s1', 'm2s2', 'm3s1']
sites = [ 'm2s1']
#label_names = ['SS', 'EB', 'CN', 'CV', 'CRS', 'Other']
label_names = ['CV', 'OTHER']
for model_idx in range(5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = label_model_CNN()
    model = model.to(device)
    model.load_state_dict(torch.load('A:/school/College_Junior/research/CNN_Tang_project/saved_models/PS_labeling_model_CNN_CV_cut' + str(model_idx)))

    for site in sites:
        dir = 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut/' + 'PS_model_set_'+ str(model_idx) + '/' + site + '/'
        make_dir_try(dir)
        val_y = np.load('../data/Processed_Tang_data/all_sites_data_prepared/New_response_data/valRsp_' + site + '.npy')
        num_neurons = val_y.shape[1]
        for neuron in range(num_neurons):
            vis_img = plt.imread("../analysis/Visualization/Shared_Core_avg_border_vis/"+site+"/" + str(neuron) + '.jpg')
            img = np.reshape(vis_img,(1,1,50,50))
            input_tensor = torch.FloatTensor(img)
            x = input_tensor.to(device)
            out = model(x)
            predict = torch.argmax(out).detach().cpu().numpy().astype(int)
            label_name = label_names[predict]
            plt.imsave(dir+str(neuron)+'_'+label_name + '.png',vis_img, cmap='gray')

Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut/PS_model_set_0/m1s1/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut/PS_model_set_0/m1s2/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut/PS_model_set_0/m1s3/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut/PS_model_set_0/m2s1/' created successfully


In [6]:
label_names = ['CV', 'OTHER']
imgs = np.load('../data/m2s1_psWS.npy')
for model_idx in range(5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = label_model_CNN()
    model = model.to(device)
    model.load_state_dict(torch.load('A:/school/College_Junior/research/CNN_Tang_project/saved_models/PS_labeling_model_CNN_CV_cut' + str(model_idx)))

    dir = 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/' + 'PS_model_set_'+ str(model_idx) + '/' + 'm2s1' + '/'
    make_dir_try(dir)
    for i, vis_img in enumerate(imgs):

        img = np.reshape(vis_img,(1,1,50,50))
        input_tensor = torch.FloatTensor(img)
        x = input_tensor.to(device)
        out = model(x)
        predict = torch.argmax(out).detach().cpu().numpy().astype(int)
        label_name = label_names[predict]
        plt.imsave(dir+str(i)+'_'+label_name + '.png',vis_img, cmap='gray')

Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/PS_model_set_0/m2s1/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/PS_model_set_1/m2s1/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/PS_model_set_2/m2s1/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/PS_model_set_3/m2s1/' created successfully
Directory 'A:/school/College_Junior/research/CNN_Tang_project/analysis/labeling/PS_CNN_CV_cut_use_ps/PS_model_set_4/m2s1/' created successfully
